### Twilight Struggle

In [1]:
from twilight_ui import *

### Game Instance

In [2]:
u = UI()
u.run()

Initalising game..
> new
Starting new game.
----- USSR Player: -----
Place starting influence.
Remaining influence: 1
Available options:
    7 Finland
   14 Austria
   16 East_Germany
   17 Poland
   18 Czechoslovakia
   19 Hungary
   20 Yugoslavia
   21 Romania
   22 Bulgaria
> m 14
Selected: Austria
----- USSR Player: -----
Place starting influence.
You have selected Austria
Remaining influence: 0
Commit your actions? (Yes/No)
> m y
----- US Player: -----
Place starting influence.
Remaining influence: 1
Available options:
    3 Canada
    4 UK
    5 Norway
    6 Sweden
    7 Finland
    8 Denmark
    9 Benelux
   10 France
   11 Spain_Portugal
   12 Italy
   13 Greece
   14 Austria
   15 West_Germany
   23 Turkey
> m 7
Selected: Finland
----- US Player: -----
Place starting influence.
You have selected Finland
Remaining influence: 0
Commit your actions? (Yes/No)
> m y
----- US Player: -----
Place additional starting influence.
Remaining influence: 2
Available options:
    3 Canada
  

> m 106
Selected: NORAD
----- USSR Player: -----
Select an action for NORAD.
Available options:
    1 RESOLVE_EVENT_FIRST
    2 INFLUENCE
    3 REALIGNMENT
    4 COUP
    5 SPACE
> m 1
Selected: RESOLVE_EVENT_FIRST
----- USSR Player: -----
Select an action for NORAD.
You have selected RESOLVE_EVENT_FIRST
Commit your actions? (Yes/No)
> m y
----- USSR Player: -----
Select an action for NORAD.
Available options:
    2 INFLUENCE
    3 REALIGNMENT
    4 COUP
    5 SPACE
> m 5
Selected: SPACE
----- USSR Player: -----
Select an action for NORAD.
You have selected SPACE
Commit your actions? (Yes/No)
> m y
Failure with roll of 5.
----- US Player: -----
Select a card in hand to play.
Available options:
    4 Duck_and_Cover
    6 The_China_Card
    8 Fidel
   10 Blockade
   12 Romanian_Abdication
   14 COMECON
   16 Warsaw_Pact_Formed
   18 Captured_Nazi_Scientist
   20 Olympic_Games
   22 Independent_Reds
   24 Indo_Pakistani_War
   26 CIA_Created
   28 Suez_Crisis
   30 Decolonization
   32 UN

TypeError: 'method' object is not iterable